In [2]:
import os
import nibabel as nib

import matplotlib.pyplot as plt

import pandas as pd

In [14]:
BASE = "/NFS/ADDiff/data/LDM"
IMGBASE = os.path.join(BASE, "Image")
stat_pheno_df = pd.read_csv(os.path.join(BASE, 'phenotype', "Longitudinal_data_BF_static.csv"))
covert_pheno_df = pd.read_csv(os.path.join(BASE, 'phenotype', "Longitudinal_data_BF_converted.csv"))
images = os.listdir(IMGBASE)

In [15]:
bad_img_list = []
with open("bad_qc_files_ldm.txt", "r") as f:
    bad_qc_files = f.readlines()
    for i in bad_qc_files:
        bad_img_list.append(i.strip().split("/")[-1])

In [16]:
def _split(x):
    return x.split("/")[-1]

In [17]:
stat_pheno_df['impath_B_split'] = stat_pheno_df['impath_B'].map(_split)
stat_pheno_df['impath_F_split'] = stat_pheno_df['impath_F'].map(_split)
covert_pheno_df['impath_B_split'] = covert_pheno_df['impath_B'].map(_split)
covert_pheno_df['impath_F_split'] = covert_pheno_df['impath_F'].map(_split)

In [18]:
good_stat_df = stat_pheno_df[~((stat_pheno_df['impath_B_split'].isin(bad_img_list)) | (stat_pheno_df['impath_F_split'].isin(bad_img_list)))]
good_convert_df = covert_pheno_df[~((covert_pheno_df['impath_B_split'].isin(bad_img_list)) | (covert_pheno_df['impath_F_split'].isin(bad_img_list)))]

In [ ]:
good_convert_df.drop(columns=['impath_B_split', 'impath_F_split'], inplace=True)
good_stat_df.drop(columns=['impath_B_split', 'impath_F_split'], inplace=True)

In [20]:
# good_convert_df.to_csv(os.path.join(BASE, 'phenotype', "Longitudinal_data_BF_converted_QC.csv"), index=False)
# good_stat_df.to_csv(os.path.join(BASE, 'phenotype', "Longitudinal_data_BF_static_QC.csv"), index=False)

## VQVAE QC

In [3]:
BASE = "/NFS/ADDiff/data/vqvae"
IMGBASE = os.path.join(BASE, "Image")
images = os.listdir(IMGBASE)

In [13]:
bad_img_list = []
with open("bad_qc_files_vqvae_train.txt", "r") as f:
    bad_qc_files = f.readlines()
    for i in bad_qc_files:
        bad_img_list.append(i.strip().split("/")[-1])

In [16]:
oas_img_list = [im for im in bad_img_list if "OAS4" in im]

In [19]:
OASPATH2 = '/NFS/MRI/OASIS_4/preprocess/cat12/Ses_2/mri/'
OASPATH3 = '/NFS/MRI/OASIS_4/preprocess/cat12/Ses_3/mri/'
OASPATH4 = '/NFS/MRI/OASIS_4/preprocess/cat12/Ses_4/mri/'
OASPATH5 = '/NFS/MRI/OASIS_4/preprocess/cat12/Ses_5/mri/'

OASPATH = [OASPATH2, OASPATH3, OASPATH4, OASPATH5]

In [ ]:
import shutil

for OPATH in OASPATH:
    for img in oas_img_list:
        if os.path.exists(os.path.join(OPATH, img)):
            print(os.path.join(BASE, "Image", 'train', img))
            shutil.copy(os.path.join(OPATH, img), os.path.join(BASE, "Image", 'train', img))
            continue

## Visual QC

In [1]:
import os
import nibabel as nib

import matplotlib.pyplot as plt

In [4]:
# IMGBASE ="/NFS/ADDiff/data/vqvae/Image/train"
IMGBASE ="/NFS/ADDiff/data/LDM/Image/"

In [5]:
imgs = os.listdir(IMGBASE)
imgs.sort()
imgs1 = imgs[:100]
imgs2 = imgs[100:200]
imgs3 = imgs[200:300]
imgs4 = imgs[300:400]
imgs5 = imgs[400:500]
imgs6 = imgs[500:600]
imgs7 = imgs[600:700]
imgs8 = imgs[700:800]
imgs9 = imgs[800:900]
imgs10 = imgs[900:1000]
imgs11 = imgs[1000:1100]
imgs12 = imgs[1100:1200]
imgs13 = imgs[1200:1300]

img_list =  [imgs1, imgs2, imgs3, imgs4, imgs5, imgs6, imgs7, imgs8, imgs9, imgs10, imgs11, imgs12, imgs13]

In [ ]:
for imgs in img_list:
    fig, axs = plt.subplots(10, 10, figsize=(20, 20))

    for i in range(10):
        for j in range(10):
            img = nib.load(os.path.join(IMGBASE, imgs[j*10+i]))
            img = img.get_fdata()
            axs[i][j].imshow(img[:,:,50], cmap='gray')
            axs[i][j].axis('off')
            axs[i][j].set_title(f'{imgs[j*10+i]}', fontsize=6)
    plt.tight_layout()
    plt.show()